In [1]:
import random
import pandas as pd
import math
import anndata
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score

In [4]:
length=100
#选取基因数量
size=20
toprange=500
crossr=0.8
mutater=0.1
generation=10
DEG = pd.read_csv('./DEG_new.csv',sep=',')
adata = anndata.read_h5ad("/home/liujingyue/wyc_20231021/single_cell_SEQ/project/chemotherapy/epicell.h5ad")
lenDEG=len(DEG)
newsize=size

/anaconda3/lib/python3.8/site-packages/anndata/compat/__init__.py:232: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(


In [15]:
def popinit(length,size,lenDEG,toprange):
    halflen=round(length/2)
    rownum=lenDEG
    list1=list(range(1,toprange,1))
    list2=list(range(rownum-toprange,rownum,1))
    popu=[]
    for i in range(size):
        chrome=[]
        chrome1=random.sample(list1,halflen)
        chrome2=random.sample(list2,halflen) 
        chrome=chrome1+chrome2
        popu.append(chrome)
    return popu  #各自从toprange范围内取前500，取值范围内随机抽取50个，两个染色体相加作为一个个体，重复形成种群

In [16]:
def getfitness(adata,popu,length,size,DEG):
    fitness=[]
    halflen=round(length/2)
    for i in range(size):
        chrome=popu[i]
        index1=chrome[:halflen-1]
        index2=chrome[halflen:]
        top=DEG.iloc[index1,[0]]
        top.columns=['genename']
        tail=DEG.iloc[index2,[0]]
        tail.columns=['genename']
        top=top['genename'].values.tolist()
        tail=tail['genename'].values.tolist()
        sc.tl.score_genes(adata,gene_list=top,score_name="malignant")
        sc.tl.score_genes(adata,gene_list=tail,score_name="normal")
        score1 = adata.obs['malignant']
        score2 = adata.obs['normal']
#         # 使用Seaborn绘制核密度估计图
#         sns.kdeplot(score1, fill=True,color="red")
#         sns.kdeplot(score2, fill=True,color="blue")
#         # 添加标题和标签
#         plt.title('Density Plot')
#         plt.xlabel('Value')
#         plt.ylabel('Density')

#         # 显示图形
#         plt.show()
#         # 将两列数据合成一个矩阵
        data_matrix = np.vstack((score1, score2)).T
    

#         # 绘制散点图
#         plt.scatter(data_matrix[:, 0], data_matrix[:, 1],s=1.5, alpha=0.2)
#         plt.xlabel('malignant')
#         plt.ylabel('normal')
#         plt.show()
        gmm = GaussianMixture(n_components=2,covariance_type='full', random_state=20240403,n_init=30,max_iter=2000,tol= 1e-2)
        gmm.fit(data_matrix)
        labels = gmm.predict(data_matrix)
        fit=0
        fit=silhouette_score(data_matrix,labels)
        fitness.append(fit)
        print(i)
    return fitness

In [17]:
def select(popu,fitness,size,newsize):
    fitcumsum=[0]
    fitcumsum+=np.cumsum(fitness).tolist()
    print(fitcumsum)
    sumnum=fitcumsum[size]
    newpopu=[]
    for i in range(newsize):
        randnum=random.uniform(0,sumnum)
        count=0
        while count<size:
            if randnum>=fitcumsum[count] and randnum<fitcumsum[count+1]:
                newpopu.append(popu[count])
                break
            count += 1
    return newpopu

In [18]:
def crossover(popu,crossr,length):
    newpopu=popu
    for i in range(0, len(popu), 2):
        if random.random()<crossr:
            cpoint=math.floor(random.random()*length)
            child1=popu[i][:cpoint]+popu[i+1][cpoint:]
            child2=popu[i+1][:cpoint]+popu[i][cpoint:]
            newpopu[i]=child1
            newpopu[i+1]=child2
    return newpopu

In [19]:
def mutate(popu,mutater,length,lenDEG):
    newpopu=popu
    rownum=round(lenDEG/2)
    halflen=round(length/2)
    for i in range(len(popu)):
        if random.random()<mutater:
            mpoint=math.floor(random.random()*length)
            print(f"变异行：{i}变异点：{mpoint}\n")
            if mpoint<halflen:
                newgene=math.floor(random.random()*lenDEG)
                while newgene in popu[i]:
                    newgene=math.floor(random.random()*lenDEG)
                print(f"变异原：{popu[i][mpoint]}变异后：{newgene}\n")
                popu[i][mpoint]=newgene
            else:
                newgene=lenDEG-math.floor(random.random()*lenDEG)
                while newgene in popu[i]:
                    newgene=lenDEG-math.floor(random.random()*lenDEG)
                print(f"变异原：{popu[i][mpoint]}变异后：{newgene}\n")
                popu[i][mpoint]=newgene  
    return newpopu

In [20]:
def genetic_algorithm(length,size,DEG,adata,mutater,crossr,generation,lenDEG,newsize,toprange):
    #初始化种群
    popu=popinit(length,size,lenDEG,toprange)
    fitness=getfitness(adata,popu,length,size,DEG)
    #输出当前最优解
    bestsolution=popu[fitness.index(max(fitness))]
    finalsolution=bestsolution
    finalfitness=max(fitness)
    print(f"初始 Best Solution = {bestsolution}, Fitness_Silhouette = {finalfitness}\n")
    
    for gen in range(generation):
    
        newpopu=popu
        
        #选择
        newpopu=select(newpopu,fitness,size,newsize)
        print(f"选择完成{gen}\n")
        
        #交叉
        newpopu=crossover(newpopu,crossr,length)
        print(f"交叉完成{gen}\n")
        
        #突变
        newpopu=mutate(newpopu,mutater,length,lenDEG)
        print(f"变异完成{gen}\n")
        
        fitness=getfitness(adata,newpopu,length,size,DEG)
        #输出当前最优解
        maxfitness=max(fitness)
        bestsolution=newpopu[fitness.index(maxfitness)]
        popu=newpopu
        
        if maxfitness>=finalfitness:
            finalfitness=maxfitness
            finalsolution=bestsolution
            print(f"新的 Generation {gen}: Best Solution = {bestsolution}, Fitness_Silhouette = {maxfitness}\n")
        else:
            print(f"稍差的 Generation {gen}: Best Solution = {bestsolution}, Fitness_Silhouette = {maxfitness}\n")
            
        print(f"循环完成{gen}\n")
    
    return finalsolution, finalfitness

In [23]:
popu=popinit(length,size,lenDEG,toprange)
fitness=getfitness(adata,popu,length,size,DEG)
#输出当前最优解
bestsolution=popu[fitness.index(max(fitness))]
finalsolution=bestsolution
finalfitness=max(fitness)
print(f"初始 Best Solution = {bestsolution}, Fitness_Silhouette = {finalfitness}\n")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45


KeyboardInterrupt: 

<h1>主程序

In [9]:
finalsolution, finalfitness=genetic_algorithm(length,size,DEG,adata,mutater,crossr,generation,lenDEG,newsize,toprange)

0
1
2
3
4
5
6
7
8
9
初始 Best Solution = [2733, 1511, 2907, 839, 1602, 5007, 5552, 3950, 6668, 6862], Fitness_Silhouette = 0.5158466313271748

[0, 0.45645354264686694, 0.8322203347185104, 1.2370392730670918, 1.6137701230135781, 2.0318775487171417, 2.5477241800443164, 2.889086332762811, 3.321031290656554, 3.6238487304211358, 4.0531160574593725]
选择完成0

交叉完成0

变异行：2变异点：1

变异原：1511变异后：6451

变异行：4变异点：5

变异原：3845变异后：2924

变异完成0

0
1
2
3
4
5
6
7
8
9
新的 Generation 0: Best Solution = [509, 2569, 3378, 529, 569, 3845, 5552, 3950, 6668, 6862], Fitness_Silhouette = 0.5519328729598252

循环完成0

[0, 0.45645354264686694, 0.9129070852937339, 1.3680852658867502, 1.9200181388465754, 2.324466135752652, 2.756411093646395, 3.0965233103041525, 3.5321170532846167, 3.873479206003111, 4.291586631706675]
选择完成1

交叉完成1

变异行：6变异点：6

变异原：6791变异后：2424

变异完成1

0
1
2
3
4
5
6
7
8
9
新的 Generation 1: Best Solution = [509, 2569, 3378, 529, 569, 3845, 5552, 3950, 6668, 6862], Fitness_Silhouette = 0.5519328729598252

循环完成1

[0,